In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install 'matplotlib == 3.6'

In [ ]:
!pip install scanpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 6.1 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8023 sha256=c43a29d948462a70bf1bc9d3f761bdc7d06a2efe500961d25454983249bbb820
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info


In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata
import os
import sys
from scipy import sparse

In [ ]:
# -- Version of the databse
cpdb_version = 'v5.0.0'

# -- Path where the input files to generate the database are located
cpdb_target_dir = os.path.join(gdrivePath+os.sep+'cellphonedb'+os.sep+'data', cpdb_version)

Creating adata and metafile

In [ ]:
import pickle
# Load object from file
with open(gdrivePath +os.sep + "data" +os.sep+'X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open(gdrivePath +os.sep + "data" +os.sep+'y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)


In [ ]:
adata_df = X_test

In [ ]:
metadata_df = pd.DataFrame(y_test)
metadata_df.rename(columns = {0:'cell_type'}, inplace = True)
metadata_df

,cell_type
0,beta
1,beta
2,acinar
3,gamma
4,acinar
...,...
2830,alpha
2831,quiescent_stellate
2832,alpha
2833,delta


In [ ]:
import anndata
adata = anndata.AnnData(adata_df)
adata

AnnData object with n_obs × n_vars = 2835 × 7514

In [ ]:
adata.obs = metadata_df
adata.obs

,cell_type
0,beta
1,beta
2,acinar
3,gamma
4,acinar
...,...
2830,alpha
2831,quiescent_stellate
2832,alpha
2833,delta


In [ ]:
vars = pd.DataFrame()
vars["gene_ids"] = adata_df.columns
vars.index = adata_df.columns
vars["feature_types"] = "Gene Expression"
vars

,gene_ids,feature_types
GCG,GCG,Gene Expression
INS,INS,Gene Expression
TTR,TTR,Gene Expression
IAPP,IAPP,Gene Expression
SST,SST,Gene Expression
...,...,...
STYK1,STYK1,Gene Expression
SNX31,SNX31,Gene Expression
GNGT1,GNGT1,Gene Expression
PATE2,PATE2,Gene Expression


In [ ]:
adata.var = vars

In [ ]:
adata.write(gdrivePath +os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+'adataTest.h5ad')

In [ ]:
metadata_df.to_csv(gdrivePath+os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+"metadataTest.tsv", sep = '\t')

------------------------------------------------

In [ ]:
meta_file_path = gdrivePath+os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+"metadataTest.tsv"
counts_file_path = gdrivePath +os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+'adataTest.h5ad'
out_path = gdrivePath +os.sep+'results/DGE/test'
cpdb_file_path = gdrivePath+os.sep+'cellphonedb'+os.sep+'data'+os.sep+cpdb_version+os.sep+'cellphonedb.zip'

In [ ]:
import anndata
adata = anndata.read_h5ad(counts_file_path)
adata

In [ ]:
adata.obs['cell_type'].values.describe()

,counts,freqs
categories,,
PSC,11,0.003880
acinar,272,0.095944
activated_stellate,57,0.020106
alpha,974,0.343563
beta,738,0.260317
delta,190,0.067019
ductal,340,0.119929
endothelial,58,0.020459
epsilon,4,0.001411


In [ ]:
df_meta = pd.DataFrame(data={'Cell':list(adata.obs.index),
                             'cell_type':[ i for i in adata.obs['cell_type']]
                            })
df_meta.set_index('Cell', inplace=True)
df_meta
#df_meta.to_csv('endometrium_example_meta.tsv', sep = '\t')

,cell_type
Cell,
0,beta
1,beta
2,acinar
3,gamma
4,acinar
...,...
2830,alpha
2831,quiescent_stellate
2832,alpha


In [ ]:
# Conver to dense matrix for Seurat
#adata.X = adata.X.toarray()
type(adata.X)

numpy.ndarray

In [ ]:
!pip install anndata2ri

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.0/221.0 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rpy2: filename=rpy2-3.5.17-cp310-cp310-linux_x86_64.whl size=331549 sha256=9a073a0fabc9896f11a50b74ed86be74ebad215963f5ad0d3831390560bfde9c
  Stored in directory: /root/.cache/pip/wheels/a0/a2/a6/9a390508fa6c600d5363cea2e146f45fb4feffa4d2ec47ba93
Successfully built rpy2
  Attempting uninstall: rpy2
    Found existing installation: rpy2 3.4.2
    Uninstalling rpy2-3.4.2:
      Successfully uninstalled rpy2-3.4.2


In [ ]:
!pip install rpy2

In [ ]:
import rpy2.rinterface_lib.callbacks
import logging
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)
import anndata2ri
anndata2ri.activate()
%load_ext rpy2.ipython

<ipython-input-22-ca75261febf2>:7: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [ ]:
from rpy2.robjects import conversion, default_converter

In [ ]:
with conversion.localconverter(default_converter):
  %R install.packages("BiocManager")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)
trying URL 'https://cran.rstudio.com/src/contrib/BiocManager_1.30.25.tar.gz'
Content type 'application/x-gzip' length 593414 bytes (579 KB)
downloaded 579 KB


The downloaded source packages are in
	‘/tmp/RtmpyjKY6k/downloaded_packages’


In [ ]:
with conversion.localconverter(default_converter):
  %R BiocManager::install("S4Vectors")

Update all/some/none? [a/s/n]: a


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)
Installing package(s) 'BiocVersion', 'S4Vectors'
also installing the dependency ‘BiocGenerics’

trying URL 'https://bioconductor.org/packages/3.20/bioc/src/contrib/BiocGenerics_0.52.0.tar.gz'
Content type 'application/x-gzip' length 57066 bytes (55 KB)
downloaded 55 KB

trying URL 'https://bioconductor.org/packages/3.20/bioc/src/contrib/BiocVersion_3.20.0.tar.gz'
Content type 'application/x-gzip' length 983 bytes
downloaded 983 bytes

trying URL 'https://bioconductor.org/packages/3.20/bioc/src/contrib/S4Vectors_0.44.0.tar.gz'
Content type 'application/x-gzip' length 1076666 bytes (1.0 MB)
downloaded 1.0 MB


The downloaded source packages are in
	‘/tmp/RtmpyjKY6k/downloaded_packages’
Old packages: 'cluster', 'openssl', 'shiny'
t

In [ ]:
with conversion.localconverter(default_converter):
  %R BiocManager::install("SingleCellExperiment")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)
Installing package(s) 'SingleCellExperiment'
also installing the dependencies ‘matrixStats’, ‘UCSC.utils’, ‘GenomeInfoDbData’, ‘abind’, ‘zlibbioc’, ‘MatrixGenerics’, ‘Biobase’, ‘IRanges’, ‘GenomeInfoDb’, ‘S4Arrays’, ‘XVector’, ‘SparseArray’, ‘SummarizedExperiment’, ‘GenomicRanges’, ‘DelayedArray’

trying URL 'https://cran.rstudio.com/src/contrib/matrixStats_1.4.1.tar.gz'
Content type 'application/x-gzip' length 213623 bytes (208 KB)
downloaded 208 KB

trying URL 'https://bioconductor.org/packages/3.20/bioc/src/contrib/UCSC.utils_1.2.0.tar.gz'
Content type 'application/x-gzip' length 236888 bytes (231 KB)
downloaded 231 KB

trying URL 'https://bioconductor.org/packages/3.20/data/annotation/src/contrib/GenomeInfoDbData_1.2.13.tar.

In [ ]:
with conversion.localconverter(default_converter):
  %R BiocManager::install("Seurat")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com
Bioconductor version 3.20 (BiocManager 1.30.25), R 4.4.2 (2024-10-31)
Installing package(s) 'Seurat'
also installing the dependencies ‘bitops’, ‘dotCall64’, ‘gtools’, ‘caTools’, ‘plyr’, ‘tensor’, ‘BH’, ‘sitmo’, ‘sp’, ‘spam’, ‘globals’, ‘listenv’, ‘parallelly’, ‘zoo’, ‘lazyeval’, ‘crosstalk’, ‘RcppTOML’, ‘here’, ‘gplots’, ‘reshape2’, ‘gridExtra’, ‘RcppArmadillo’, ‘spatstat.data’, ‘spatstat.univar’, ‘spatstat.random’, ‘spatstat.utils’, ‘spatstat.sparse’, ‘goftest’, ‘deldir’, ‘polyclip’, ‘FNN’, ‘dqrng’, ‘SeuratObject’, ‘cowplot’, ‘fastDummies’, ‘fitdistrplus’, ‘future’, ‘future.apply’, ‘ggrepel’, ‘ggridges’, ‘ica’, ‘igraph’, ‘irlba’, ‘leiden’, ‘lmtest’, ‘patchwork’, ‘pbapply’, ‘plotly’, ‘png’, ‘progressr’, ‘RANN’, ‘RcppAnnoy’, ‘RcppHNSW’, ‘reticulate’, ‘ROCR’, ‘RSpectra’, ‘Rtsne’, ‘scattermore’, ‘sct

In [ ]:
with conversion.localconverter(default_converter):
  %R library(Seurat)


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

Loading required package: SeuratObject
Loading required package: sp

Attaching package: ‘SeuratObject’

The following objects are masked from ‘package:base’:

    intersect, t



In [ ]:
with conversion.localconverter(default_converter):
  %R library(SingleCellExperiment)

Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: ‘MatrixGenerics’

The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges, ro

In [ ]:
with conversion.localconverter(default_converter):
  %R -i adata
  adata

In [ ]:
with conversion.localconverter(default_converter):
  %R so = as.Seurat(adata, counts = "X", data = "X")

In [ ]:
with conversion.localconverter(default_converter):
  %R Idents(so) = so$cell_type

In [ ]:
## OPTION 1 - compute DEGs for all cell types
## Extract DEGs for each cell_type
with conversion.localconverter(default_converter):
  %R DEGs <- FindAllMarkers(so, test.use = 'LR', verbose = F, only.pos = T, random.seed = 1, logfc.threshold = 1, min.pct = 0.1, return.thresh = 0.05) #logfc.threshold =0.2

In addition: Warning messages:
1: glm.fit: fitted probabilities numerically 0 or 1 occurred 
2: glm.fit: algorithm did not converge 
3: glm.fit: fitted probabilities numerically 0 or 1 occurred 
4: glm.fit: fitted probabilities numerically 0 or 1 occurred 
5: glm.fit: fitted probabilities numerically 0 or 1 occurred 
6: glm.fit: algorithm did not converge 
7: glm.fit: fitted probabilities numerically 0 or 1 occurred 


In [ ]:
with conversion.localconverter(default_converter):
  %R -o DEGs

In [ ]:
out_path

'gdrive/My Drive/Sevgi Phd/conference(second_tik)/results/DGE/test'

In [ ]:
import pickle

# Save object to file
with open(gdrivePath +os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+"DEGs.pkl", 'wb') as f:
    pickle.dump(DEGs, f)

In [ ]:
DEGs.to_csv(gdrivePath +os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+'DEGs.tsv', index=False, sep='\t')

In [ ]:
DEGs.head()

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
RCN3,2.055261e-20,6.608999,1.000,0.066,1.544323e-16,PSC,RCN3
DCN,8.463581e-19,7.155906,0.909,0.029,6.359535e-15,PSC,DCN
LUM,5.332860e-18,6.500879,0.909,0.032,4.007111e-14,PSC,LUM
MMP2,7.222255e-16,6.394307,0.909,0.039,5.426802e-12,PSC,MMP2
MXRA8,2.587146e-15,6.442274,0.909,0.025,1.943982e-11,PSC,MXRA8


In [ ]:
cond1 = DEGs['p_val_adj'] < 0.05
cond2 = DEGs['avg_log2FC'] > 1 #0.1
mask = [all(tup) for tup in zip(cond1, cond2)]
fDEGs = DEGs[mask]

In [ ]:
fDEGs.head()

,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
RCN3,2.055261e-20,6.608999,1.000,0.066,1.544323e-16,PSC,RCN3
DCN,8.463581e-19,7.155906,0.909,0.029,6.359535e-15,PSC,DCN
LUM,5.332860e-18,6.500879,0.909,0.032,4.007111e-14,PSC,LUM
MMP2,7.222255e-16,6.394307,0.909,0.039,5.426802e-12,PSC,MMP2
MXRA8,2.587146e-15,6.442274,0.909,0.025,1.943982e-11,PSC,MXRA8


In [ ]:
with open(gdrivePath +os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+"DEGs_mask.pkl", 'wb') as f:
    pickle.dump(fDEGs, f)

In [ ]:
# 1st column = cluster; 2nd column = gene
fDEGs = fDEGs[['cluster', 'gene', 'p_val_adj', 'p_val', 'avg_log2FC', 'pct.1', 'pct.2']]
fDEGs.to_csv(gdrivePath +os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+'DEGs_mask.tsv', index=False, sep='\t')

In [ ]:
fDEGs.head()

,cluster,gene,p_val_adj,p_val,avg_log2FC,pct.1,pct.2
RCN3,PSC,RCN3,1.544323e-16,2.055261e-20,6.608999,1.000,0.066
DCN,PSC,DCN,6.359535e-15,8.463581e-19,7.155906,0.909,0.029
LUM,PSC,LUM,4.007111e-14,5.332860e-18,6.500879,0.909,0.032
MMP2,PSC,MMP2,5.426802e-12,7.222255e-16,6.394307,0.909,0.039
MXRA8,PSC,MXRA8,1.943982e-11,2.587146e-15,6.442274,0.909,0.025


In [ ]:
fDEGs.to_csv(gdrivePath +os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+'DEGs_mask.txt', index=False, sep='\t')

In [ ]:
# 1st column = cluster; 2nd column = gene
cellphonedb_DEGs = fDEGs[['cluster', 'gene', 'avg_log2FC','p_val', 'p_val_adj', 'pct.1', 'pct.2']]
cellphonedb_DEGs.rename(columns={'cluster': 'cell_type', 'avg_log2FC': 'logFC','p_val': 'P.Value','p_val_adj': 'adj.P.Val'}, inplace=True)

cellphonedb_DEGs.to_csv(gdrivePath +os.sep+"results"+os.sep+"DGE"+os.sep+"test"+os.sep+'Cellphonedb_DEGs_mask.tsv', index=False, sep='\t')

In [ ]:
cellphonedb_DEGs.head()

,cell_type,gene,logFC,P.Value,adj.P.Val,pct.1,pct.2
RCN3,PSC,RCN3,6.608999,2.055261e-20,1.544323e-16,1.000,0.066
DCN,PSC,DCN,7.155906,8.463581e-19,6.359535e-15,0.909,0.029
LUM,PSC,LUM,6.500879,5.332860e-18,4.007111e-14,0.909,0.032
MMP2,PSC,MMP2,6.394307,7.222255e-16,5.426802e-12,0.909,0.039
MXRA8,PSC,MXRA8,6.442274,2.587146e-15,1.943982e-11,0.909,0.025
